In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
from nltk.tag import StanfordNERTagger
from collections import Counter
import sys
import os
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [2]:
def write_clusters(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

def analyse_location(content):
    st = StanfordNERTagger('/data2/Stanford_tools/stanford-ner-2017-06-09/classifiers/english.all.3class.distsim.crf.ser.gz',
                       '/data2/Stanford_tools/stanford-ner-2017-06-09/stanford-ner.jar',  encoding='utf-8')
    location_words = []
    print "Analyzing Location words, this may take a while..."
    i = 1
    for sen in content:
        tokenized_text = nltk.word_tokenize(sen)
        classified_text = st.tag(tokenized_text)
        for word in classified_text:
            if word[1] == 'LOCATION':
                location_words.append(word[0])
        sys.stdout.write("\r" + "%d/%d\r" %(i,len(content)))
        sys.stdout.flush()
        i+=1
    location_freq = Counter(location_words)
    print location_freq
    return location_freq
# location_words =  location_freq.most_common(64):
#[(u'Nagpur', 21), (u'Nagercoil', 14), (u'Tirunelveli', 14), (u'Surat', 13), (u'Jamnagar', 12), (u'Kolhapur', 12), (u'Patna', 11), (u'Road', 11), (u'Pondicherry', 10), (u'Noida', 9), (u'Jaipur', 9), (u'Bhuj', 9), (u'Pune', 8), (u'Lucknow', 8), (u'Madurai', 8), (u'Palakkad', 8), (u'Jabalpur', 8), (u'Ahmedabad', 7), (u'Gandhinagar', 7), (u'Kanyakumari', 7), (u'Karad', 7), (u'Raipur', 7), (u'Satara', 6), (u'Delhi', 6), (u'city', 6), (u'Aurangabad', 6), (u'Bhubaneswar', 6), (u'Lanson', 6), (u'Salem', 6), (u'Chennai', 5), (u'Navsari', 5), (u'&', 5), (u'Kerala', 5), (u'Dehradun', 5), (u'Varchha', 5), (u'Shillong', 5), (u'Nashik', 5), (u'Guwahati', 5), (u'Gandhi', 5), (u'Ajmer', 5), (u'City', 5), (u'MIDC', 5), (u'Coimbatore', 5), (u'Mumbai', 5), (u'Rourkela', 4), (u'Kanpur', 4), (u'Dibrugarh', 4), (u'Kozhikode', 4), (u'Udaipur', 4), (u'Vasai', 4), (u'Palghar', 4), (u'Khandawara', 4), (u'Andheri', 4), (u'Dealership', 4), (u'Jodhpur', 4), (u'Bangalore', 4), (u'Gandhidham', 4), (u'Ahmadabad', 4), (u'Dindigul', 4), (u'Nagar', 4), (u'Kolkata', 4), (u'Trichy', 4), (u'Ambikapur', 4), (u'Margaon', 4)]


def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df

    

# Reading Raw text

In [3]:
question_class = 'q6'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()

with open('..//data//q6_original.txt') as f:
        content_original = f.readlines()
content_original = [x.strip() for x in content_original]
f.close()

length of content 1377


# Extract locations

In [4]:
#location_freq = analyse_location(content_original)
location_freq = Counter({u'Nagpur': 21, u'Nagercoil': 14, u'Tirunelveli': 14, u'Surat': 13, u'Jamnagar': 12, u'Kolhapur': 12, u'Patna': 11, u'Road': 11, u'Pondicherry': 10, u'Noida': 9, u'Jaipur': 9, u'Bhuj': 9, u'Pune': 8, u'Lucknow': 8, u'Madurai': 8, u'Palakkad': 8, u'Jabalpur': 8, u'Ahmedabad': 7, u'Gandhinagar': 7, u'Kanyakumari': 7, u'Karad': 7, u'Raipur': 7, u'Satara': 6, u'Delhi': 6, u'city': 6, u'Aurangabad': 6, u'Bhubaneswar': 6, u'Lanson': 6, u'Salem': 6, u'Chennai': 5, u'Navsari': 5, u'&': 5, u'Kerala': 5, u'Dehradun': 5, u'Varchha': 5, u'Shillong': 5, u'Nashik': 5, u'Guwahati': 5, u'Gandhi': 5, u'Ajmer': 5, u'City': 5, u'MIDC': 5, u'Coimbatore': 5, u'Mumbai': 5, u'Rourkela': 4, u'Kanpur': 4, u'Dibrugarh': 4, u'Kozhikode': 4, u'Udaipur': 4, u'Vasai': 4, u'Palghar': 4, u'Khandawara': 4, u'Andheri': 4, u'Dealership': 4, u'Jodhpur': 4, u'Bangalore': 4, u'Gandhidham': 4, u'Ahmadabad': 4, u'Dindigul': 4, u'Nagar': 4, u'Kolkata': 4, u'Trichy': 4, u'Ambikapur': 4, u'Margaon': 4, u'Tuticorin': 3, u'Nagarkovil': 3, u'Banaras': 3, u'Mansarovar': 3, u'Borivali': 3, u'Nanded': 3, u'Durgapur': 3, u'Vashin': 3, u'Karkala': 3, u'Malappuram': 3, u'Yavatmal': 3, u'Gujrat': 3, u'Porbandar': 3, u'Nandurbar': 3, u'Chhattisgarh': 3, u'Hosur': 3, u'Bokaro': 3, u'Bombay': 3, u'Rajkot-Jamnagar': 3, u'Faizabad': 3, u'Thanjavur': 3, u'Toyota': 3, u'Rishikesh': 3, u'Sharayu': 3, u'Mysore': 3, u'Varaccha': 3, u'Theni': 3, u'Pathankot': 3, u'nagar': 3, u'Bihar': 3, u'Madgaon': 3, u'Cuttack': 3, u'Haldwani': 3, u'Highland': 3, u'Karaikudi': 3, u'Hospet': 3, u'Rudrapur': 3, u'Kannur': 3, u'Annamalai': 2, u'Nalasopara': 2, u'Batala': 2, u'Tejpur': 2, u'Shinrai': 2, u'Deharadun': 2, u'Pala': 2, u'Kanchipuram': 2, u'Velachery': 2, u'Alwar': 2, u'Malegaon': 2, u'Indore': 2, u'Vagholi': 2, u'Morvi': 2, u'Shilong': 2, u'Worli': 2, u'Kalyan': 2, u'Mira': 2, u'Kasargod': 2, u'Pathanamthitta': 2, u'Tambaram': 2, u'Kashipur': 2, u'Chembur': 2, u'Raigarh': 2, u'North': 2, u'Shirdi': 2, u'Mallur': 2, u'Pudukkottai': 2, u'Rajahmundry': 2, u'Vardha': 2, u'Gondiya': 2, u'Thutukudi': 2, u'Gwalior': 2, u'Bikaner': 2, u'Dindugul': 2, u'Vilaspur': 2, u'Ludhiana': 2, u'Parbhani': 2, u'Idukki': 2, u'Thalacherry': 2, u'Buldhana': 2, u'Vashi': 2, u'Rajasthan': 2, u'Varkkala': 2, u'Thoothukudi': 2, u'Khar': 2, u'Banaskata': 2, u'Kumbakonam': 2, u'Patia': 2, u'Mehsana': 2, u'Dahisar': 2, u'Thane': 2, u'Rampur': 2, u'Avinashi': 2, u'Bellary': 2, u'Alibaug': 2, u'Agra': 2, u'Vellapanchavati': 2, u'Nippon': 2, u'Amravati': 2, u'Bhivadi': 2, u'Nagarcoil': 2, u'Bhubaneshwar': 2, u'Naka': 2, u'Kundapura': 2, u'Varachha': 2, u'Thrissur': 2, u'Mandya': 2, u'Morabi': 2, u'Hadapsar': 2, u'Vishakhapatnam': 1, u'Nagor': 1, u'Marthandam': 1, u'Varasa': 1, u'Jilla': 1, u'Ushmanabad': 1, u'Hazira': 1, u'Koyampattur': 1, u'Bengal': 1, u'Itanagar': 1, u'Cherthala': 1, u'Meerut': 1, u'Rajivghandi': 1, u'Chidawa': 1, u'Narooli': 1, u'Kochi': 1, u'Avadi': 1, u'Khandwa': 1, u'Patiala': 1, u'Sadra': 1, u'Hadwani': 1, u'Ramghat': 1, u'Balladur': 1, u'Tirupathur': 1, u'Jogeshwari': 1, u'Vadakkara': 1, u'Ramnathpuram': 1, u'Chalakkudi': 1, u'Chakkara': 1, u'Singanallur': 1, u'Jalandhar': 1, u'Muradabad': 1, u'Raigunj': 1, u'Sitapur': 1, u'Kushaiguda': 1, u'Pudhiya': 1, u'Burshanpur': 1, u'Kozhilcode': 1, u'Waggle': 1, u'East': 1, u'Annanagar': 1, u'Kankavali': 1, u'Bhavan': 1, u'Churchgate': 1, u'Kananad': 1, u'Angaadi': 1, u'Kasaragoda': 1, u'Bhoisar': 1, u'Anagul': 1, u'Palampur': 1, u'Rudra': 1, u'Thirupattur': 1, u'Pur': 1, u'Bhasmat': 1, u'Palighar': 1, u'Barista': 1, u'Namakal': 1, u'Usmanabad': 1, u'Trichur': 1, u'Arthur': 1, u'Pritampura': 1, u'Green': 1, u'Gour': 1, u'Pattambhi': 1, u'Thallacherry': 1, u'MC': 1, u'Gorakhpur': 1, u'Manjiri': 1, u'Pali': 1, u'Anand': 1, u'Ashapur': 1, u'Kotharud': 1, u'Bokhara': 1, u'Adajan': 1, u'Mirut': 1, u'Ropar': 1, u'Sharapur-Rajpur': 1, u'Naganiroad': 1, u'Trishul': 1, u'Irinjalakuda': 1, u'Bardoli': 1, u'Bhosari': 1, u'Kamti': 1, u'Nakodar': 1, u'Khuhadi': 1, u'Tirur': 1, u'Shilphata': 1, u'Adoor': 1, u'Baitul': 1, u'Baitun': 1, u'Muzafferpur': 1, u'Tamluk': 1, u'Yelanka': 1, u'Bazapur': 1, u'Etawah': 1, u'Pimpri': 1, u'Rajirapalli': 1, u'Beguni': 1, u'Kanoor': 1, u'Parle': 1, u'Gomathi': 1, u'Sitabadi': 1, u'Rajarampuri': 1, u'Pavas': 1, u'Chattarpur': 1, u'Palam': 1, u'Virar': 1, u'Pondani': 1, u'Latoor': 1, u"''Sona": 1, u'Nasik': 1, u'Kadalur': 1, u'Aathur': 1, u'Raigad': 1, u'Kottayam': 1, u'Nilambur': 1, u'Warli': 1, u'Vatapattamam': 1, u'Hadapasar': 1, u'Mahu': 1, u'Tamiya': 1, u'Durg': 1, u'Kollour': 1, u'Faizbabad': 1, u'Banpur': 1, u'Rahta': 1, u'Boisar': 1, u'Vadkanjery': 1, u'Amarkantan-Jabalpur': 1, u'Uppala': 1, u'Maphasa': 1, u'Hajirta': 1, u'White': 1, u'Gargya': 1, u'Kamlanagar': 1, u'Ramnagar': 1, u'Katappana': 1, u'Bardhhaman': 1, u'Morbee': 1, u'Kaswasd': 1, u'Tomak': 1, u'Manjeri': 1, u'Trichendur': 1, u'Sikandar': 1, u'Khammam': 1, u'Vaniyambadi': 1, u'Ahmednagar': 1, u'Vellore': 1, u'Telitparamba': 1, u'Pendha': 1, u'Pimpalgaon': 1, u'Kharghar': 1, u'Ponto': 1, u'Indirapuram': 1, u'Bigone': 1, u'Shankermate': 1, u'Noapara': 1, u'Sahib': 1, u'Mahanagarpalica': 1, u'Irriki': 1, u'Motipuram': 1, u'Which': 1, u'Sajanpur': 1, u'Ozargaon': 1, u'Bomvaom': 1, u'Ajira': 1, u'Chatainaka': 1, u'Chandpur': 1, u'Addapal': 1, u'Eragipalam': 1, u'Varadha': 1, u'Nandhanam': 1, u'highway': 1, u'Town': 1, u'Aknoor': 1, u'Faridkot': 1, u'Taluka': 1, u'Yavat': 1, u'Maharashtra': 1, u'Colaba': 1, u'Raypur': 1, u'Adichira': 1, u'Chatapur': 1, u'Kolara': 1, u'Bilaspur': 1, u'Whitefield': 1, u'Virudhunagar': 1, u'Talasory': 1, u'Barboli': 1, u'Patel': 1, u'south': 1, u'Bhimavaram': 1, u'Asansol': 1, u'kanpur': 1, u'Kukkur': 1, u'Gomatinagar': 1, u'Dhavada': 1, u'Kankarwar': 1, u'Dwarka': 1, u'District': 1, u'Semnad': 1, u'Gangapur': 1, u'Angamally': 1, u'Mansoor': 1, u'Kolaghat': 1, u'Taratala': 1, u'Kurla': 1, u'Hojai': 1, u'Pottai': 1, u'Mallapuram': 1, u'Vairna': 1, u'Puducherry': 1, u'Elathur': 1, u'Chemanna': 1, u'Ardiya': 1, u'Panikoili': 1, u'Thodibuia': 1, u'Linkingroad': 1, u'Bardi': 1, u'Pogal': 1, u'Haryana': 1, u'Allapi': 1, u'Wardha': 1, u'Bandra': 1, u'Varacha': 1, u'Mahakali': 1, u'Khetpur': 1, u'Gangawa': 1, u'Baramati': 1, u'Azamgarh': 1, u'Chindawara': 1, u'Kinadhykidav': 1, u'Daruheda': 1, u'Kohianady': 1, u'Mhape': 1, u'Hardaloo': 1, u'Karkal': 1, u'Kutulli': 1, u'Mardhgao': 1, u'Rajapalayam': 1, u'Latur': 1, u'Dhanbad': 1, u'Varachaa': 1, u'Tara': 1, u'Tilichaerry': 1, u'Balabhgarh': 1, u'Hingewadi': 1, u'Shiroor': 1, u'Nagarkoveli': 1, u'Bhopal': 1, u'Kadaloure': 1, u'Dhanabad': 1, u'Masrur': 1, u'Renukot': 1, u'Nadakkavu': 1, u'Patana': 1, u'Vashind': 1, u'Ekkattuthangal': 1, u'Paanoor': 1, u'Chikkudi': 1, u'Jila': 1, u'Azartganj': 1, u'Podi': 1, u'Palankadi': 1, u'Kanniyakumari': 1, u'Kumbhakonam': 1, u'Basistha': 1, u'Bagalkote': 1, u'Neral': 1, u'Kasaragod': 1, u'Nishandine': 1, u'Sengalpatta': 1, u'Kandivali': 1, u'Kuddapah': 1, u'Mandoli': 1, u'Malda': 1, u'Thirumangai': 1, u'Sangrur': 1, u'Srirampur': 1, u'Hoogly': 1, u'Kamothe': 1, u'Kalahandi': 1, u'Nagaon': 1, u'Ramanathapuram': 1, u'Mundarkkaya': 1, u'Shirai': 1, u'Sonnur': 1, u'Grand': 1, u'Abids': 1, u'Brahrampur': 1, u'Orissa': 1, u'Gurgaon': 1, u'KR': 1, u'Dombivli': 1, u'Burhanpur': 1, u'Tinsukia': 1, u'Malapuram': 1, u'Sellum': 1, u'Bewar': 1, u'Dharmapuri': 1, u'Bathinja': 1, u'Panthalam': 1, u'Cidko': 1, u'Jalana': 1, u'Khola': 1, u'Kangra': 1, u'Kanooz': 1, u'Jhansi': 1, u'Perambattur': 1, u'Ramnade': 1, u'Solapur': 1, u'Thuraipakkam': 1, u'Sagarpada': 1, u'Thiravalla': 1, u'Pattukottai': 1, u'Vadodara': 1, u'Thiroor': 1, u'Guiney': 1, u'Naroda': 1, u'Lahore': 1, u'Temple': 1, u'Vijayawada': 1, u'Tilwara': 1, u'Tumkur': 1, u'Sindhudurg': 1, u'Maniktala': 1, u'Kamatee': 1, u'Tata': 1, u'Dhagya': 1, u'Calicut': 1, u'Kodampakkam': 1, u'Suryapet': 1, u'Udhampur': 1, u'Assam': 1, u'Peraij': 1, u'Rajiv': 1, u'Dinajpur': 1, u'Bodhguya': 1, u'Vapi': 1, u'Kerla': 1, u'Field': 1, u'Ghaziabad': 1, u'Bedbihar': 1, u'Vattapalav': 1, u'Gajuwaka': 1, u'Bhubheneshwar': 1, u'Sibsagar': 1, u'Kumar': 1, u'Vadalur': 1, u'Kodadha': 1, u'Koturekara': 1, u'Yelhanka': 1, u'Kakkannad': 1, u'Chikhalthana': 1, u'Bilayat': 1, u'Araria': 1, u'Antara': 1, u'Kanchirapally': 1, u'Vorli': 1, u'Jalgaon': 1, u'Koria': 1, u'Airport': 1, u'Plaghar': 1, u'Palanpur': 1, u'Jakat': 1, u'Madhgev': 1, u'Vijay': 1, u'Harsha': 1, u'Nimkathana': 1, u'Kolam': 1, u'Panaji': 1, u'Sambalpur': 1, u'Chinchwad': 1, u'Sriparampattur': 1, u'Vasant': 1, u'Srigonda': 1, u'Kottarakkara': 1, u'Teribram': 1, u'Sundar': 1, u'Trangala': 1, u'Irutto': 1, u'Tanjour': 1, u'Varanasi': 1, u'Chitradurga': 1, u'Chandrapur': 1, u'Trivandrum': 1, u'Udaynagar': 1, u'Om': 1, u'ROPAR': 1, u'Vagali': 1, u'Haldia': 1, u'Gangnagar': 1, u'Koduvila': 1, u'Podigaru': 1, u'Vile': 1, u'Bhoiser': 1, u'Ooty': 1, u'Jharsuguda': 1, u'Visakhapatnam': 1, u'Majestic': 1, u'Vatika': 1, u'Rajpura': 1, u'Tapasvi': 1, u'Kakud': 1, u'Karamandal': 1, u'India': 1, u'Punjab': 1, u'Kamrej': 1, u'Thirupur': 1, u'Koyambedu': 1, u'Metro': 1, u'Mawoo': 1, u'Nabha': 1, u'Shrirampur': 1, u'Kurukshetra': 1, u'Marthalli': 1, u'Morbi': 1, u'Govandi': 1, u'Anshavad': 1, u'Hebbala': 1, u'Ernakulum': 1, u'Alappuzha': 1, u'Kenya': 1, u'Chariali': 1, u'Ramnadu': 1, u'Kanjipuram': 1, u'Magadi': 1, u'Howrah': 1, u'Gadag': 1, u'Chandranagar': 1, u'Ratlam': 1, u'Gurueayoor': 1, u'Aram': 1, u'Velur': 1, u'Manjeeri': 1, u'Phypumu': 1, u'Iringali': 1, u'Kharadi': 1, u'Poonamalle': 1, u'Attavalli': 1, u'Nausara': 1, u'Kagadi': 1, u'Mahapa': 1, u'Payyanur': 1, u'South': 1, u'Tumbthakatte': 1, u'Nagarcooil': 1, u'Allahabad': 1, u'Revdegaon': 1, u'Barashi': 1, u'Nizan': 1, u'S.B': 1, u'Ankaleshwar': 1, u'Kondodi': 1, u'Vaishali': 1, u'Mathuman': 1, u'Punnarlu': 1, u'Makhmallabad': 1, u'Sulur': 1, u'Hindustan': 1, u'Makar': 1, u'Thamarashery': 1, u'Kollam': 1, u'Vasna': 1, u'Liva': 1, u'Madakpura': 1, u'Highway': 1, u'Chandigarh': 1, u'Sreerampur': 1, u'Alanaputtam': 1, u'Maninagar': 1, u'Dadar': 1, u'Dharamapuri': 1, u'Shirur': 1, u'Great': 1, u'Sundargarde': 1, u'Metapallam': 1, u'Vaikom': 1, u'Gummidipoondi': 1, u'Gungarh': 1, u'Thalippurampil': 1, u'Nerul': 1, u'Cuddalore': 1, u'PATNA': 1, u'Kalappalam': 1, u'Mogappair': 1, u'Rajkot': 1, u'Burmunda': 1, u'Ashok': 1, u'Srivilliputhur': 1, u'Nirander': 1, u'Tirupatur': 1, u'Nanavati': 1, u'Village': 1, u'Bhandup': 1, u'Jubilihills': 1})
location_words = location_freq.most_common()
location_words = [word[0] for word in location_words if word[0] not in \
                  ['Road','city', '&','City','Dealership','Toyota','Highway','highway', 'south','District','district',\
                  'Which','Town', 'Airport', '']]

def process_locations(content_original):
    location_extracted = []  # location words
    doc_location = []   # original sentences
    location_idx = []   # index in content_original
    for i, sen in enumerate(content_original):
        location_list = [word for word in sen.split() if word in location_words]
        location_list = list(set(location_list))       
        if location_list != []:
            location_extracted.append(location_list)
            doc_location.append(sen)
            location_idx.append(i)
    return location_extracted, doc_location, location_idx
        
location_extracted, doc_location, location_idx =process_locations(content_original)     
print "location words extracted"
print "Number of sentences containing location words: %d" %len(location_extracted)

location words extracted
Number of sentences containing location words: 574


## Cluster location words

In [5]:
import heapq
import os
    
def filter_location(test_corpus, df):  # assuming each review contain one aspect
    print "filtering location words"
    for xth, doc in enumerate(test_corpus):
        if len(doc)>1:
            df_words = [df[word] for word in doc]
#             df_words = []
#             for word in doc:
#                 if df[word] == 1:
#                     df_words.append(1000)
#                 else:
#                     df_words.append(df[word])
#             idx =  heapq.nsmallest(1, xrange(len(df_words)), key=df_words.__getitem__) #use smallest df to represent
            idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__) #use largest df to represent
            test_corpus[xth] = [doc[ith] for ith in idx]
    return test_corpus

def write_file(corpus, idx_list, word, count):
    if not os.path.isdir("cluster/location"):
            os.mkdir("cluster/location")
    f1 = open('cluster/location/%d_%s.txt' %(count, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()
    
def location_category(df_list, location_clean, corpus):
    if (not os.path.isdir("cluster")):
        os.mkdir("cluster")
    clustered_idx = []
    major_location = [word for word in df_list if df_list[word] > 1]
    for word in major_location:
        idx_set = []
        for idx, doc in enumerate(location_clean):
            if word in doc:
                idx_set.append(idx)
        count = len(idx_set)
        clustered_idx = clustered_idx + idx_set
        write_file(corpus, idx_set, word, count)
    clustered_idx = set(clustered_idx)
    print "No. of clustered locations:%d"%len(clustered_idx)
    unclustered_idx = list(set(range(len(location_clean))) - clustered_idx)
    write_file(corpus, unclustered_idx, 'others', len(unclustered_idx))
    print "No. of others locations:%d"%len(unclustered_idx)

        
location_clean = filter_location(location_extracted, location_freq)
df = df_count(location_clean)
df_dic = dict(df.most_common())

location_category(df_dic, location_clean,doc_location)
print "location clustered"


filtering location words
No. of clustered locations:334
No. of others locations:240
location clustered


# Process non-location comments

In [6]:
content_nonlocation = [content[i] for i in range(len(content)) if i not in location_idx]
print "non-location sentences:%d"%len(content_nonlocation)       
# split comment with multi-sentence into multi-comments
content_new = []
for comment in content_nonlocation:
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
print 'length of new content', len(content_new)
# print content_new
    

non-location sentences:803
length of new content 1755


## Extract keywords

In [7]:
pos_new = nltk.pos_tag(nltk.word_tokenize("it is 10 km far away from city it should be nearby city within 2 3 km . "))
print pos_new

[('it', 'PRP'), ('is', 'VBZ'), ('10', 'CD'), ('km', 'NN'), ('far', 'RB'), ('away', 'RB'), ('from', 'IN'), ('city', 'NN'), ('it', 'PRP'), ('should', 'MD'), ('be', 'VB'), ('nearby', 'JJ'), ('city', 'NN'), ('within', 'IN'), ('2', 'CD'), ('3', 'CD'), ('km', 'NN'), ('.', '.')]


In [8]:
stemmer = PorterStemmer()
print stemmer.stem("company")

compani


In [9]:

def rule_q1(sen, ne, location_words):
    clean_ne = list(set(ne))
#     remove_words = ["improv", "custom", "servic", "peopl","person","facil","avail","good",\
#                     "center","centr","car", "dealership", "vehicl", "toyota", "problem","work", "much",\
#                    "thing", "possibl","need"]   #stemmed
    remove_words = ['dealership','locat', 'servic', 'center','km','car','place', 'custom', 'toyota', \
                    'facil','time', 'problem', 'centr','home', 'side','vehicl','peopl', 'lot', 'compani',\
                   'day',]
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = ["pick", 'pickup', 'drop', 'insid','outsid'] + location_words  #stemmed
    clean_ne = clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]
    
    # rules to merge keywords:
    if 'out side' in sen:
        clean_ne.append('outside')
    if 'pickup' in clean_ne:
        clean_ne[clean_ne.index('pickup')] = 'pick'
        
    clean_ne = list(set(clean_ne))
    return clean_ne


def process_corpus(x1, pos_tags, general_stop, location_words):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
                #stemming
                for counter,word in enumerate(nn_list):
                    nn_list[counter] = stemmer.stem(word)
                #apply rule
                nn_list = rule_q1(sen, nn_list, location_words)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS']
                  
doc1, doc2, doc3 = process_corpus(content_new, pos_tags, stop_words, location_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'signle sentence with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'single sentence without nn', len(doc3)
print nn_extracted

signle sentence with nn 1290
no comments 23
single sentence without nn 442
[['jodhpur'], ['panchayat', 'road'], ['road'], [u'condit', 'road'], ['board'], [u'issu', 'traffic'], ['area'], [u'staff'], ['area', 'tv', 'etc', 'cushion', u'chair', u'wait'], ['aundh'], ['showroom'], [u'deliveri', u'charg'], [u'branch'], [u'clean'], [u'entri', 'highway', 'gate'], ['town', u'citi', u'distanc', u'branch', u'area'], ['traffic', 'branch'], ['shop', u'henc'], [u'sunshin'], ['borivali'], ['sinhgad', 'road'], ['vishakhapatnam'], ['district'], [u'hadups', 'road'], [u'citi'], [u'bypass', 'road', 'area'], ['need', u'citi', 'showroom', 'outskirt', u'farmer'], ['wagholi', 'shirur'], [u'citi', 'nashik'], [u'citi', 'area'], ['showroom'], [u'park', 'road'], ['water', u'hole', 'session', u'lodg', 'road'], ['weather', 'fogg', 'winter', 'way', 'board'], [u'branch', 'area'], ['howrah'], [u'close', u'open'], [u'respond'], ['taratala'], ['hingewadi'], ['town'], [u'citi'], [u'citi', u'insid'], ['mud', u'road'], [u'r

In [10]:

df = df_count(nn_extracted)
print df

Counter({u'citi': 234, 'area': 148, 'road': 97, 'drop': 61, 'showroom': 54, u'insid': 53, 'pick': 43, 'town': 42, 'highway': 39, u'outsid': 35, 'traffic': 30, u'bu': 29, 'way': 25, u'distanc': 22, u'henc': 19, 'space': 18, u'charg': 17, 'work': 16, u'villag': 15, u'issu': 15, u'transport': 15, u'hous': 14, u'improv': 14, u'kilomet': 13, u'thing': 13, 'shop': 13, 'toll': 11, u'offic': 11, 'food': 11, 'need': 11, 'stop': 11, u'conveni': 10, u'part': 10, 'pickup': 10, 'board': 10, u'branch': 10, u'park': 9, u'arrang': 9, u'staff': 9, 'water': 8, u'bypass': 8, 'auto': 8, 'workshop': 8, u'driver': 8, 'bit': 8, 'canteen': 8, u'hour': 7, 'person': 7, u'govern': 6, 'market': 6, 'tax': 6, u'number': 6, 'front': 6, 'district': 6, u'noth': 6, u'increas': 6, 'maruti': 6, 'dealer': 6, 'patna': 6, u'roadsid': 5, u'sourc': 5, 'tea': 5, 'airport': 5, u'chang': 5, 'outer': 5, u'someth': 5, 'hyundai': 5, 'station': 5, 'sale': 5, u'local': 5, 'outlet': 5, u'morn': 5, 'rajendra': 5, u'everyth': 5, u'resid

# Decide category for each comment

In [11]:
# import heapq
# import os
# def filter_rule_q1(doc, xth, test_corpus, original_corpus = doc_nn):
#     apply_rule = False
#     # print original_corpus[xth]
#     # rules (priority from high to low)
#     if 'app' in doc:
#         test_corpus[xth] = ['app']
#         print "rule_app applied"
#         apply_rule = True
#     elif 'without_appointment' in doc:
#         test_corpus[xth] = ['without_appointment']
#         print "rule_without_appointment applied"
#         apply_rule = True
        
#     return apply_rule
    
# def filter_ne(test_corpus, df):  # assuming each review contain one aspect
#     for xth, doc in enumerate(test_corpus):
#         if len(doc)>1:
#             apply_rule = filter_rule_q1(doc, xth, test_corpus)
#             if not apply_rule:
#                 df_words = [df[word] for word in doc]
#                 idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__)
#                 test_corpus[xth] = [doc[ith] for ith in idx]
#     return test_corpus

# nn_clean = filter_ne(nn_extracted, df)
# df = df_count(nn_clean)
# dict_map = dict(df.most_common())
# print df

# def write_file(corpus, idx_list, word):
#     if not os.path.isdir("cluster/%s" %word):
#             os.mkdir("cluster/%s" %word)
#     f1 = open('cluster/%s/%s_comment.txt' % (word, word), 'w+')
#     for idx in idx_list:
#         f1.write('%s\n' %corpus[idx])
#     f1.close()
    
    
# def main_category(df_list, nn_clean, corpus):
#     if (not os.path.isdir("cluster")):
#         os.mkdir("cluster")
#     name_list = {}
#     clustered_index = []
#     major_list = [word for word in df_list if df_list[word]>2]
#     print "majot list:\n", major_list
#     for word in major_list:
#         idx_set = []
#         for idx, doc in enumerate(nn_clean):
#             if word in doc:
#                 idx_set.append(idx)
#         write_file(doc_nn, idx_set, word)
#         name_list[word] = idx_set
#         clustered_index = clustered_index + idx_set
#     clustered_index = set(clustered_index)
#     unclustered_index = list(set(range(len(nn_clean))) - clustered_index)
#     name_list['others'] = unclustered_index
#     write_file(doc_nn, unclustered_index, 'others')
#     return name_list

# name_list = main_category(dict_map, nn_clean,doc_nn)
# # print name_list['call']

## clustering using LSI

In [12]:
from gensim import corpora
from gensim.models import TfidfModel
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
from sklearn.metrics.pairwise import cosine_similarity

dictionary = corpora.Dictionary(nn_extracted)
corpus_gensim = [dictionary.doc2bow(doc) for doc in nn_extracted]
tfidf = TfidfModel(corpus_gensim)
corpus_tfidf = tfidf[corpus_gensim]
lsi = LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)
lsi_proj = lsi[corpus_tfidf]
lsi_index = MatrixSimilarity(lsi[corpus_tfidf])
similarity_matrix = np.array([lsi_index[lsi[corpus_tfidf[i]]]
                                for i in range(len(nn_extracted))])


Using TensorFlow backend.


In [13]:
print similarity_matrix

def f(x):
    return (x+1)/2
f = np.vectorize(f, otypes=[np.float])
similarity_matrix = f(similarity_matrix)     # convert similarity_matrix from -1~1 to 0~1
print similarity_matrix

[[  1.00000000e+00   1.11391917e-02  -3.11717833e-03 ...,   9.24258977e-02
    2.50012660e-03  -4.14082743e-02]
 [  1.11391898e-02   1.00000000e+00   9.89447773e-01 ...,  -3.57378391e-03
   -5.34445047e-04   3.94051149e-03]
 [ -3.11717810e-03   9.89447713e-01   1.00000012e+00 ...,   1.38030387e-03
    7.84471631e-05  -1.88279152e-03]
 ..., 
 [  9.24258903e-02  -3.57378367e-03   1.38030387e-03 ...,   1.00000000e+00
   -3.40678744e-05   1.39136054e-02]
 [  2.50012637e-03  -5.34452498e-04   7.84397125e-05 ...,  -3.40678453e-05
    1.00000012e+00   8.19447339e-01]
 [ -4.14082780e-02   3.94051149e-03  -1.88279897e-03 ...,   1.39136016e-02
    8.19447339e-01   9.99999940e-01]]
[[ 1.          0.5055696   0.49844141 ...,  0.54621295  0.50125006
   0.47929586]
 [ 0.50556959  1.          0.99472389 ...,  0.49821311  0.49973278
   0.50197026]
 [ 0.49844141  0.99472386  1.00000006 ...,  0.50069015  0.50003922
   0.4990586 ]
 ..., 
 [ 0.54621295  0.49821311  0.50069015 ...,  1.          0.49998297


In [14]:
from sklearn.cluster import SpectralClustering
import os

n_clusters_ = 10
sc = SpectralClustering(n_clusters=n_clusters_, affinity='precomputed').fit(similarity_matrix)
labels = sc.labels_
print('Estimated number of clusters: %d' % n_clusters_)
print len(labels)

def write_file(n_clusters_, labels, doc_nn, nn_extracted):
    if (not os.path.isdir("cluster_%s" %question_class)):
        os.mkdir("cluster_%s" %question_class)
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (doc_nn[idx], nn_extracted[idx]))
    f1.close()
    print("clusters written to folder: cluster_%s" %question_class)
    
write_file(n_clusters_, labels, doc_nn, nn_extracted)

Estimated number of clusters: 10
1290
clusters written to folder: cluster_q6
